# Фајн тунинг Open AI модела

Овај нотебук је заснован на тренутним упутствима из [Fine Tuning](https://platform.openai.com/docs/guides/fine-tuning?WT.mc_id=academic-105485-koreyst) документације Open AI.

Фајн тунинг побољшава перформансе основних модела за вашу апликацију поновним тренирањем са додатним подацима и контекстом релевантним за тај конкретан случај употребе или сценарио. Имајте у виду да технике инжењеринга упита као што су _few shot learning_ и _retrieval augmented generation_ омогућавају да побољшате подразумевани упит релевантним подацима како бисте побољшали квалитет. Међутим, ови приступи су ограничени максималном величином прозора токена циљаног основног модела.

Фајн тунингом ефективно поново тренирамо сам модел са потребним подацима (што нам омогућава да користимо много више примера него што може да стане у максимални прозор токена) - и распоређујемо _прилагођену_ верзију модела која више не мора да има примере обезбеђене у време инференције. Ово не само да побољшава ефикасност дизајна нашег упита (имамо већу флексибилност у коришћењу прозора токена за друге ствари), већ потенцијално и смањује трошкове (смањењем броја токена које морамо послати моделу у време инференције).

Фајн тунинг има 4 корака:
1. Припремите податке за тренирање и отпремите их.
1. Покрените посао тренирања да бисте добили фајн тунирани модел.
1. Процените фајн тунирани модел и по потреби поновите ради квалитета.
1. Распоредите фајн тунирани модел за инференцију када будете задовољни.

Имајте у виду да не подржавају сви основни модели фајн тунинг - [проверите OpenAI документацију](https://platform.openai.com/docs/guides/fine-tuning/what-models-can-be-fine-tuned?WT.mc_id=academic-105485-koreyst) за најновије информације. Такође можете фајн тунирати раније фајн тунирани модел. У овом туторијалу ћемо користити `gpt-35-turbo` као наш циљни основни модел за фајн тунинг.

---


### Корак 1.1: Припремите свој скуп података

Хајде да направимо чатбот који ће вам помоћи да разумете периодни систем елемената тако што ће одговарати на питања о елементу у виду лимерика. У _овом_ једноставном туторијалу, направићемо само скуп података за тренирање модела са неколико примера одговора који показују очекивани формат података. У стварном случају употребе, потребно је направити скуп података са много више примера. Такође, можда ћете моћи да користите отворени скуп података (за вашу примену) ако постоји, и да га преформатирате за употребу у финој обуци.

Пошто се фокусирамо на `gpt-35-turbo` и тражимо одговор у једном кораку (chat completion), можемо направити примере користећи [предложени формат](https://platform.openai.com/docs/guides/fine-tuning/preparing-your-dataset?WT.mc_id=academic-105485-koreyst) који одражава захтеве OpenAI за chat completion. Ако очекујете разговор у више корака, користили бисте [формат примера за више корака](https://platform.openai.com/docs/guides/fine-tuning/multi-turn-chat-examples?WT.mc_id=academic-105485-koreyst) који укључује параметар `weight` да означи које поруке треба користити (или не) у процесу фине обуке.

За наш туторијал користићемо једноставнији формат за један корак. Податке су у [jsonl формату](https://jsonlines.org/?WT.mc_id=academic-105485-koreyst) са по једним записом по линији, сваки представљен као JSON објекат. Испод је приказано 2 записа као пример - погледајте [training-data.jsonl](../../../../../18-fine-tuning/python/openai/training-data.jsonl) за цео пример (10 примера) који ћемо користити у овом туторијалу за фину обуку. **Напомена:** Сваки запис _мора_ бити дефинисан у једној линији (не сме бити раздвојен преко више линија као што је уобичајено у форматираном JSON фајлу)

```json
{ "messages": [{"role": "system", "content": "Elle is a factual chatbot that answers questions about elements in the periodic table with a limerick"}, {"role": "user", "content": "Tell me about Gallium"}, {"role": "assistant", "content": "Gallium, oh gallium, so light - Melts in your hand, oh what a sight - At 86 degrees - Its liquid with ease - And in semiconductors, it's out of sight"}]}
{ "messages": [{"role": "system", "content": "Elle is a factual chatbot that answers questions about elements in the periodic table with a limerick"}, {"role": "user", "content": "Tell me about Hydrogen"}, {"role": "assistant", "content": "Hydrogen, the first in the line - The lightest of all, so divine - It's in water, you see - And in stars, it's the key - The universe's most common sign"}]}
```

У стварном случају употребе биће вам потребан много већи скуп примера за добре резултате - компромис ће бити између квалитета одговора и времена/трошкова за фину обуку. Ми користимо мали скуп да бисмо брзо завршили фину обуку и илустровали процес. Погледајте [овaj пример из OpenAI Cookbook-а](https://github.com/openai/openai-cookbook/blob/main/examples/How_to_finetune_chat_models.ipynb?WT.mc_id=academic-105485-koreyst) за сложенији туторијал фине обуке.


---

### Корак 1.2 Отпремите свој скуп података

Отпремите податке користећи Files API [као што је описано овде](https://platform.openai.com/docs/guides/fine-tuning/upload-a-training-file). Имајте у виду да да бисте покренули овај код, морате прво да урадите следеће кораке:
 - Инсталирали сте `openai` Python пакет (проверите да користите верзију >=0.28.0 за најновије функције)
 - Поставили сте `OPENAI_API_KEY` променљиву окружења на ваш OpenAI API кључ
За више информација, погледајте [Водич за подешавање](./../../../00-course-setup/02-setup-local.md?WT.mc_id=academic-105485-koreyst) обезбеђен за курс.

Сада покрените код да бисте креирали фајл за отпремање из вашег локалног JSONL фајла.


In [24]:
from openai import OpenAI
client = OpenAI()

ft_file = client.files.create(
  file=open("./training-data.jsonl", "rb"),
  purpose="fine-tune"
)

print(ft_file)
print("Training File ID: " + ft_file.id)

FileObject(id='file-JdAJcagdOTG6ACNlFWzuzmyV', bytes=4021, created_at=1715566183, filename='training-data.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)
Training File ID: file-JdAJcagdOTG6ACNlFWzuzmyV


---

### Корак 2.1: Креирајте посао фино подешавање помоћу SDK-а


In [25]:
from openai import OpenAI
client = OpenAI()

ft_filejob = client.fine_tuning.jobs.create(
  training_file=ft_file.id, 
  model="gpt-3.5-turbo"
)

print(ft_filejob)
print("Fine-tuning Job ID: " + ft_filejob.id)

FineTuningJob(id='ftjob-Usfb9RjasncaZ5Cjbuh1XSCh', created_at=1715566184, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-EZ6ag0n0S6Zm8eV9BSWKmE6l', result_files=[], seed=830529052, status='validating_files', trained_tokens=None, training_file='file-JdAJcagdOTG6ACNlFWzuzmyV', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix=None)
Fine-tuning Job ID: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh


---

### Корак 2.2: Проверите статус посла

Ево неколико ствари које можете урадити са `client.fine_tuning.jobs` API-јем:
- `client.fine_tuning.jobs.list(limit=<n>)` - Прикажи последњих n послова фино подешавања
- `client.fine_tuning.jobs.retrieve(<job_id>)` - Добиј детаље о одређеном послу фино подешавања
- `client.fine_tuning.jobs.cancel(<job_id>)` - Откажи посао фино подешавања
- `client.fine_tuning.jobs.list_events(fine_tuning_job_id=<job_id>, limit=<b>)` - Прикажи до n догађаја из посла
- `client.fine_tuning.jobs.create(model="gpt-35-turbo", training_file="your-training-file.jsonl", ...)`

Први корак у процесу је _валидација тренинг фајла_ како би се осигурало да су подаци у исправном формату.


In [26]:
from openai import OpenAI
client = OpenAI()

# List 10 fine-tuning jobs
client.fine_tuning.jobs.list(limit=10)

# Retrieve the state of a fine-tune
client.fine_tuning.jobs.retrieve(ft_filejob.id)

# List up to 10 events from a fine-tuning job
client.fine_tuning.jobs.list_events(fine_tuning_job_id=ft_filejob.id, limit=10)

SyncCursorPage[FineTuningJobEvent](data=[FineTuningJobEvent(id='ftevent-GkWiDgZmOsuv4q5cSTEGscY6', created_at=1715566184, level='info', message='Validating training file: file-JdAJcagdOTG6ACNlFWzuzmyV', object='fine_tuning.job.event', data={}, type='message'), FineTuningJobEvent(id='ftevent-3899xdVTO3LN7Q7LkKLMJUnb', created_at=1715566184, level='info', message='Created fine-tuning job: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh', object='fine_tuning.job.event', data={}, type='message')], object='list', has_more=False)

In [30]:
# Once the training data is validated
# Track the job status to see if it is running and when it is complete
from openai import OpenAI
client = OpenAI()

response = client.fine_tuning.jobs.retrieve(ft_filejob.id)

print("Job ID:", response.id)
print("Status:", response.status)
print("Trained Tokens:", response.trained_tokens)

Job ID: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh
Status: running
Trained Tokens: None


---

### Корак 2.3: Пратите догађаје да бисте пратили напредак


In [44]:
# You can also track progress in a more granular way by checking for events
# Refresh this code till you get the `The job has successfully completed` message
response = client.fine_tuning.jobs.list_events(ft_filejob.id)

events = response.data
events.reverse()

for event in events:
    print(event.message)

Step 85/100: training loss=0.14
Step 86/100: training loss=0.00
Step 87/100: training loss=0.00
Step 88/100: training loss=0.07
Step 89/100: training loss=0.00
Step 90/100: training loss=0.00
Step 91/100: training loss=0.00
Step 92/100: training loss=0.00
Step 93/100: training loss=0.00
Step 94/100: training loss=0.00
Step 95/100: training loss=0.08
Step 96/100: training loss=0.05
Step 97/100: training loss=0.00
Step 98/100: training loss=0.00
Step 99/100: training loss=0.00
Step 100/100: training loss=0.00
Checkpoint created at step 80 with Snapshot ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWyyF2:ckpt-step-80
Checkpoint created at step 90 with Snapshot ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWyzhK:ckpt-step-90
New fine-tuned model created: ft:gpt-3.5-turbo-0125:bitnbot::9OFWzNjz
The job has successfully completed


### Корак 2.4: Погледајте статус у OpenAI контролној табли


Такође можете проверити статус тако што ћете посетити OpenAI вебсајт и истражити одељак _Фино подешавање_ платформе. Ово ће вам показати статус тренутног посла, као и омогућити праћење историје претходних извршења послова. На овом снимку екрана можете видети да је претходно извршење било неуспешно, а друго извршење успешно. За контекст, ово се догодило када је прво извршење користило JSON фајл са неправилно форматираним записима - након исправке, друго извршење је успешно завршено и модел је постао доступан за коришћење.

![Fine-tuning job status](../../../../../translated_images/fine-tuned-model-status.563271727bf7bfba7e3f73a201f8712fae3cea1c08f7c7f12ca469c06d234122.sr.png)


Такође можете видети статусне поруке и метрике померањем надоле у визуелној контролној табли као што је приказано:

| Поруке | Метрике |
|:---|:---|
| ![Messages](../../../../../translated_images/fine-tuned-messages-panel.4ed0c2da5ea1313b3a706a66f66bf5007c379cd9219cfb74cb30c0b04b90c4c8.sr.png) |  ![Metrics](../../../../../translated_images/fine-tuned-metrics-panel.700d7e4995a652299584ab181536a6cfb67691a897a518b6c7a2aa0a17f1a30d.sr.png)|


---

### Корак 3.1: Преузмите ИД и тестирате фино подешени модел у коду


In [46]:
# Retrieve the identity of the fine-tuned model once ready
response = client.fine_tuning.jobs.retrieve(ft_filejob.id)
fine_tuned_model_id = response.fine_tuned_model
print("Fine-tuned Model ID:", fine_tuned_model_id)

Fine-tuned Model ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWzNjz


In [47]:
# You can then use that model to generate completions from the SDK as shown
# Or you can load that model into the OpenAI Playground (in the UI) to validate it from there.
from openai import OpenAI
client = OpenAI()

completion = client.chat.completions.create(
  model=fine_tuned_model_id,
  messages=[
    {"role": "system", "content": "You are Elle, a factual chatbot that answers questions about elements in the periodic table with a limerick"},
    {"role": "user", "content": "Tell me about Strontium"},
  ]
)
print(completion.choices[0].message)

ChatCompletionMessage(content="Strontium, a metal so bright - It's in fireworks, a dazzling sight - It's in bones, you see - And in tea, it's the key - It's the fortieth, so pure, that's the right", role='assistant', function_call=None, tool_calls=None)


---

### Корак 3.2: Учитајте и тестирате фино подешени модел у Playground-у

Сада можете тестирати фино подешени модел на два начина. Прво, можете посетити Playground и користити падајући мени Models да изаберете ваш ново фино подешени модел са листе опција. Друга опција је да користите опцију "Playground" приказану у панелу за фино подешавање (погледајте снимак екрана изнад) која покреће следећи _упоредни_ приказ који показује основни и фино подешени модел један поред другог за брзу процену.

![Fine-tuning job status](../../../../../translated_images/fine-tuned-playground-compare.56e06f0ad8922016497d39ced3d84ea296eec89073503f2bf346ec9718f913b5.sr.png)

Једноставно унесите системски контекст који сте користили у вашим подацима за обуку и поставите ваше тест питање. Приметићете да су обе стране ажуриране са идентичним контекстом и питањем. Покрените упоређивање и видећете разлику у излазима између њих. _Обратите пажњу како фино подешени модел приказује одговор у формату који сте навели у вашим примерима, док основни модел једноставно прати системски упит_.

![Fine-tuning job status](../../../../../translated_images/fine-tuned-playground-launch.5a26495c983c6350c227e05700a47a89002d132949a56fa4ff37f266ebe997b2.sr.png)

Приметићете да упоређивање такође пружа број токена за сваки модел и време потребно за извршење инференције. **Овај конкретан пример је поједностављен и има за циљ да покаже процес, али не одражава стварни скуп података или сценарио**. Можда ћете приметити да оба узорка показују исти број токена (системски контекст и кориснички упит су идентични) при чему фино подешени модел троши више времена за инференцију (прилагођени модел).

У стварним сценаријима нећете користити овако једноставан пример, већ ћете фино подешавати на стварним подацима (нпр. каталог производа за корисничку подршку) где ће квалитет одговора бити много очигледнији. У _том_ контексту, добијање еквивалентног квалитета одговора са основним моделом захтеваће више прилагођеног инжењеринга упита што ће повећати коришћење токена и потенцијално време обраде за инференцију. _Да бисте то испробали, погледајте примере фино подешавања у OpenAI Cookbook-у да бисте почели._

---


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**Одрицање од одговорности**:
Овај документ је преведен коришћењем AI услуге за превођење [Co-op Translator](https://github.com/Azure/co-op-translator). Иако се трудимо да превод буде тачан, имајте у виду да аутоматски преводи могу садржати грешке или нетачности. Оригинални документ на његовом изворном језику треба сматрати ауторитетним извором. За критичне информације препоручује се професионални људски превод. Нисмо одговорни за било каква неспоразума или погрешна тумачења која произилазе из коришћења овог превода.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
